In [8]:
import requests
import pandas as pd
import datetime

# Set the API key
API_KEY = "07557ae0579026fc394b0d4989de8360"

# Set the location
lat = 40.712783
lon = -74.005941

# Set the start and end dates
start = "1357016400"
end = "1640966400"

# start = "2023-01-01T00:00:00Z"
# end = "2023-02-01T00:00:00Z"



# Make the API call
params = {
    "lat": lat,
    "lon": lon,
    "start": start,
    "end": end,
    "appid": API_KEY
}

response = requests.get("http://api.openweathermap.org/data/2.5/air_pollution/history", params=params)

# Check the response status code
if response.status_code == 200:
    # The request was successful
    air_quality_data = response.json()

    data_points = air_quality_data['list']

    # Create a DataFrame from the data points
    df = pd.DataFrame(data_points)

    # Expand components dictionary into separate columns
    components_df = pd.DataFrame(df["components"].tolist())


    # Combine the original DataFrame and the expanded components DataFrame
    result_df = pd.concat([df["main"], df["dt"], components_df], axis=1)

    result_df['main'] = result_df['main'].apply(lambda x: x['aqi'])
    result_df.rename(columns={"main": "aqi"}, inplace=True)

else:
    # The request failed
    print("Error:", response.status_code)

In [9]:
result_df.head()

,aqi,dt,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,5,1606266000,1815.80,211.00,79.51,0.0,7.87,63.81,90.35,14.06
1,5,1606269600,2056.12,239.61,80.20,0.0,8.35,77.15,110.21,15.07
2,5,1606273200,2189.64,257.49,79.51,0.0,8.94,86.94,126.06,16.47
3,5,1606276800,2349.85,278.95,80.88,0.0,9.30,96.19,141.36,15.58
4,5,1606280400,2109.53,246.76,76.08,0.0,7.99,83.11,129.25,9.12


In [10]:
# result_df['dt'] = result_df['dt'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y/%m/%d'))
result_df['dt'] = result_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%Y/%m/%d'))
# datetime.datetime.fromtimestamp(x)

In [11]:
result_df.head()

,aqi,dt,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,5,2020/11/24,1815.80,211.00,79.51,0.0,7.87,63.81,90.35,14.06
1,5,2020/11/24,2056.12,239.61,80.20,0.0,8.35,77.15,110.21,15.07
2,5,2020/11/24,2189.64,257.49,79.51,0.0,8.94,86.94,126.06,16.47
3,5,2020/11/24,2349.85,278.95,80.88,0.0,9.30,96.19,141.36,15.58
4,5,2020/11/25,2109.53,246.76,76.08,0.0,7.99,83.11,129.25,9.12


In [12]:
result_df.to_csv('./Complementary/air_quality_data.csv', index=False)